# Bricklayer Robot

In [1]:
# Extensions: magic to avoid re-loading modified .py modules all the time
%load_ext autoreload
%autoreload 2

# Bookkeeping
import sys
import shutil
import os as os
import matplotlib.pyplot as plt

# Custom project classes and functions
from iiwa import *
from Trajectory import *
from TrajectoryBuilder import *

# Some paths
kProjectDir = os.getcwd()
print("Working in directory ", kProjectDir)

# Hack: move our model packages to the manipulation/ folder to allow Drake to find them
shutil.copyfile(kProjectDir + "/models/brick.dmd.yaml", kProjectDir + "/../manipulation/manipulation/models/brick.dmd.yaml")
shutil.copyfile(kProjectDir + "/models/real_brick.sdf", kProjectDir + "/../manipulation/manipulation/models/real_brick.sdf")
shutil.copyfile(kProjectDir + "/models/ground_model.sdf", kProjectDir + "/../manipulation/manipulation/models/ground_model.sdf")

# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


Working in directory  /home/nikita/MIT_6_4212/project


In [2]:
# System configs
# Number of bricks
kNumberOfBricks =  10# @param

# Brick warehouse config
kWhLocation = np.array([2.5, 2.5, 0.15]) # location of the warehouse (corner), # @param
kWhSize = np.array([5, 5, 5]) # size of the warehouse grid in cells, # @param
if (kWhSize.prod() < kNumberOfBricks):
    assert False, "The warehouse is too small to fit all bricks"

# Brick source
X_WBrickSource = RigidTransform(np.array([0.5, 0.5, 0.05])) # @param

In [3]:
# Instantiate the robot (TODO: remove it from here)
ilyich = IIWA_Ilyich(meshcat, kNumberOfBricks)

In [4]:
# Build trajectories
trj_builder = TrajectoryBuilder(ilyich, X_WBrickSource)
trj_builder.gen_initial_traj()

targets = [np.array([-0.3, 0.3, 0.05])]
z = 0
y = 0
for i in range (1, 6):
    #
    if i % 2 == 0:
        y = 0
        z = z + 0.1
    else:
        y = y + 0.170
    #
    targets.append(np.array([-0.3, 0.3 + y, 0.05 + z]))

targets.append(np.array([0.0, 0.4, 0.05]))
z = 0
y = 0
for i in range (1, 6):
    #
    if i % 3 == 0:
        y = 0
        z = z + 0.1
    else:
        y = y + 0.170
    #
    targets.append(np.array([0.0, 0.4 + y, 0.05 + z]))

for i in range(0, kNumberOfBricks):
    # Compute destination and orientation
    #if i%2:
    #    X_WBrickTarget = RigidTransform(np.array([-0.3, -0.3, 0.35]))
    #else:
    #    X_WBrickTarget = RigidTransform(np.array([-0.3, 0.3, 0.35]))
    X_WBrickTarget = RigidTransform(targets[i])

    # Generate trajectory
    trj_builder.gen_grab_brick_traj(i)
    trj_builder.gen_move_to_place_traj(X_WBrickTarget, i)
    trj_builder.gen_place_brick_traj(X_WBrickTarget, i)
    trj_builder.gen_return_to_source_traj(X_WBrickTarget, i)

# Speed-up a bit
trj_builder.get_trajectories().slow_down(0.5)

#trj_builder.get_trajectories().dump_trajectories([True, True, False, True, False, False])

kUseIK = True # controller to use @param
if kUseIK:
    # Use IK
    # Make trajectories for iiwa with IK
    trj_builder.solve_IK()
    # Form iiwa trajectories with IK
    traj, finger_traj = trj_builder.get_trajectories().form_iiwa_traj_q()
else:
    # Use PseudoInverse controller
    # Form iiwa trajectories with IK
    traj, finger_traj = trj_builder.get_trajectories().form_iiwa_traj()

IK solver: failed to solve for  0  positions


#

In [5]:
# Create robot
ilyich = IIWA_Ilyich(meshcat, kNumberOfBricks, [traj, finger_traj])

# Create default context
context = ilyich.CreateDefaultContext()

# Put all bricks in the warehouse
ilyich.put_bricks_in_warehouse(context, kWhLocation, kWhSize, kNumberOfBricks)

In [6]:
#
# Move
#
# Move to calibration point and calibrate
sim, ctx = ilyich.work(None, context, sim_duration=trj_builder.get_trajectories().get_breakpoints()[0])
q0 = ilyich.get_q0(ctx)
print("calibration q0= ", q0)

# Continue for all bricks
for i in range(0, kNumberOfBricks):
    # Reset integrator with calibrated value
    ilyich.reset_integrator(ctx, q0)

    # Lock prev brick at target (if any)
    if (i > 1):
        ilyich.lock_brick(context, i - 2)

    # Teleport the next new brick from the warehouse and unlock it
    ilyich.move_brick(context, X_WBrickSource, i)
    ilyich.unlock_brick(context, i)

    # Manipulate the brick to target
    sim, ctx = ilyich.work(sim, ctx, sim_duration=trj_builder.get_trajectories().get_breakpoints()[i+1])

calibration q0=  [-1.93518422 -1.18923627  1.91867454 -0.89358737  1.0201813   1.57142632
  1.99789302]


KeyboardInterrupt: 